# Import Libraries

In [1]:
from swarmintelligence import GeneticAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/GA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_gens': 100, 'mr': 0.2, 'cr': 0.6}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
ga_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                ga_evaluation_results['image_name'].append(image_name)
                ga_evaluation_results['thresholds'].append(threshold)
                ga_evaluation_results['fitness_function'].append(fitness_function)
                ga_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # GA
                    ga = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    # Evaluation Metrics
                    st = time.time()
                    populations, best_thresholds = ga.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(ga.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(ga.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                ga_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                ga_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                ga_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                ga_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                ga_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                ga_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                ga_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                ga_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                ga_evaluation_results['Regions'].append(mean_regions)
                ga_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                ga_evaluation_results['Fitness'].append(fitness_values)
                ga_evaluation_results['MSE'].append(list_mse)
                ga_evaluation_results['RMSE'].append(list_rmse)
                ga_evaluation_results['PSNR'].append(list_psnr)
                ga_evaluation_results['SSIM'].append(list_ssim)
                ga_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1934.4573906804328
1941.4837700488795
1946.2917721379335
1941.610703109028
1922.971261099441
1894.3601429605005
1942.555611867098
1948.4377685101895
1906.8392974398944
1929.7463641842446
1939.5710018050092
1910.5622859119937
1946.2991641548901
1947.2684741692067
1926.027181443953
1933.204971061633
1948.0316801449706
1938.0948460145623
1930.307212397082
1930.048976257838
1916.6522110649973
1927.3075407236568
1948.6765950132303
1948.0332066619258
1946.6978412301487
1925.209299778911
1938.5126157612412
1944.673738971629
1918.5062457615086
1931.234696872901
Mean Fitness = 1933.4557955746307
	 AirplaneF16.tiff | Threshold =  3
1981.6751208278931
2009.8702308419417
2009.631928600354
1989.7409047684055
2000.918323328524
2020.274316051356
2005.9798799352775
1973.6625188991588
2016.6228279549268
2004.4711418475417
1987.3560097688605
2018.379151447194
1981.6679635241449
2014.335890702966
1990.6854111743244
2007.9376517006244
1993.6979919567298
1965.61133601220

3218.946626072141
Mean Fitness = 3194.2668436052895
	 Mandrill.tiff | Threshold =  2
1534.2009913977763
1546.8180050055314
1512.139311179254
1502.1423388025914
1532.0953696974125
1546.8180050055314
1541.9936954612394
1535.9794058555178
1539.7362544455518
1520.1215271367591
1493.127342288611
1524.6200201006357
1523.9859878721352
1541.1953089420117
1541.9936954612394
1542.9152894766157
1530.4316943635085
1547.4988210301722
1540.0818735083947
1423.8124244658313
1541.5132884700947
1543.4602493166822
1530.661490818459
1536.0045834213338
1538.3525564044426
1500.7157958201285
1460.708131821178
1541.9327888752366
1534.8196935129504
1544.1216259683902
Mean Fitness = 1526.4665855308408
	 Mandrill.tiff | Threshold =  3
1602.3795800174614
1552.496972538049
1615.9741505727504
1608.0360377043417
1626.5270428912727
1594.8637753674961
1602.6485482928044
1634.8607593896597
1597.6184201677918
1602.6674383908987
1612.7490901331148
1616.5683142385383
1619.4324912397085
1626.0433572768666
1635.738233951096

4164.192354910156
4166.82699238524
4062.3476558911143
4134.639805943854
4194.896848543729
4169.088342890075
Mean Fitness = 4158.330033303654


## Visualize Results using DataFrame

In [7]:
ga_evaluation_results_df =pd.DataFrame(ga_evaluation_results)
ga_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[113, 173]",1.149447,1933.455796,2299.776033,47.892543,14.537597,0.776005,0.938165,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.0620169639587402, 1.1775734424591064, 1.097...","[1934.4573906804328, 1941.4837700488795, 1946....","[2485.7418670654297, 2228.623664855957, 2319.6...","[49.85721479450522, 47.20830080458263, 48.1628...","[14.176243337182427, 14.650436230629744, 14.47...","[0.7790655933507594, 0.7905592257312759, 0.791...","[0.9355683707649983, 0.9386011026010189, 0.939..."
1,AirplaneF16.tiff,3,otsu,max,"[85, 140, 186]",1.200358,2002.246276,1891.609539,43.334357,15.428002,0.771950,0.950249,"[[85, 255, 255, 186, 186, 186, 255, 255, 255, ...","[1.1818561553955078, 1.035250186920166, 1.2042...","[1981.6751208278931, 2009.8702308419417, 2009....","[2282.4610481262207, 2056.637893676758, 2013.1...","[47.775109085445536, 45.350169720484594, 44.86...","[14.546769861525654, 14.999225274237748, 15.09...","[0.7949364127066803, 0.7961038732510053, 0.815...","[0.9458594594232917, 0.946139562317601, 0.9502..."
2,AirplaneF16.tiff,4,otsu,max,"[74, 115, 152, 188]",1.237562,2040.524560,1707.892098,41.150179,15.885581,0.782276,0.957063,"[[74, 255, 255, 188, 188, 188, 255, 255, 255, ...","[1.2302932739257812, 1.310821771621704, 1.2312...","[2045.0819465896823, 2023.4383753562524, 2035....","[1653.97904586792, 1994.3845481872559, 1514.99...","[40.66914119904575, 44.65853275900649, 38.9229...","[15.945503576595932, 15.132714601250042, 16.32...","[0.7730517097495522, 0.7952388233802861, 0.754...","[0.9589273621870964, 0.950770547471092, 0.9619..."
3,AirplaneF16.tiff,5,otsu,max,"[61, 104, 137, 169, 196]",1.275041,2064.616074,1419.146023,37.108553,16.897896,0.778083,0.964001,"[[104, 255, 196, 196, 196, 196, 196, 196, 196,...","[1.4677627086639404, 1.2738635540008545, 1.340...","[2083.0850183824687, 2049.4995295520594, 2045....","[850.8446083068848, 1798.4973793029785, 1926.8...","[29.169240790717964, 42.40869461918132, 43.895...","[18.83230109733989, 15.581705515491237, 15.282...","[0.7415712300107136, 0.8072400512057968, 0.810...","[0.9743357638586554, 0.9587532478775351, 0.955..."
4,Lena.png,2,otsu,max,"[96, 154]",1.217836,1934.918731,2864.264096,52.925175,13.749639,0.611063,0.879259,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.1883492469787598, 1.3492591381072998, 1.209...","[1946.8986738442045, 1949.5680596828797, 1891....","[2335.514991760254, 2015.7555084228516, 2099.0...","[48.32716618797603, 44.89716592862908, 45.8150...","[14.446977014822354, 15.086425056053105, 14.91...","[0.598844265212429, 0.6243423647901378, 0.6371...","[0.886265707770219, 0.8903712901865879, 0.8758..."
5,Lena.png,3,otsu,max,"[80, 124, 172]",1.238385,2083.094310,1623.399645,39.863737,16.199124,0.680163,0.921751,"[[172, 172, 172, 172, 172, 172, 172, 172, 172,...","[1.19631028175354, 1.2308387756347656, 1.20330...","[2029.6473269661947, 2106.128726195147, 2073.0...","[1910.8174743652344, 1442.0192184448242, 1508....","[43.712898260870716, 37.97392814082873, 38.842...","[15.318611566291631, 16.541093124061927, 16.34...","[0.6394782091243391, 0.6978196760201597, 0.695...","[0.8971716632669055, 0.92442509529077, 0.91900..."
6,Lena.png,4,otsu,max,"[69, 107, 142, 179]",1.229294,2146.156209,1201.770992,33.986966,17.652131,0.718429,0.942806,"[[179, 179, 179, 179, 179, 179, 179, 179, 179,...","[1.387894868850708, 1.0897853374481201, 1.3588...","[2170.358635951585, 2151.582226042376, 2164.32...","[1549.1809616088867, 926.1125755310059, 1109.4...","[39.35963619761858, 30.4320977839354, 33.30787...","[16.229782097078868, 18.464165794124625, 17.67...","[0.6824948942169049, 0.744776508530165, 0.7325...","[0.9396694466950991, 0.945930176250424, 0.9503..."
7,Lena.png,5,otsu,max,"[63, 96, 122, 157, 189]",1.378169,2170.43419

# Save results

In [8]:
pickle.dump(ga_evaluation_results_df, open('results/evaluation/GA/GA_otsu_df.pkl', 'wb'))
pickle.dump(ga_evaluation_results, open('results/evaluation/GA/GA_otsu_dict.pkl', 'wb'))